In [1]:
import requests
import pandas as pd
import collections

In [2]:
r = requests.get('http://dataservices.imf.org/RESsT/SDMX_JSON.svc/Dataflow/')

In [5]:
def DataflowMethod():
    r = requests.get('http://dataservices.imf.org/REST/SDMX_JSON.svc/Dataflow/')
    IDs = pd.DataFrame(r.json()['Structure']['Dataflows']['Dataflow'])['KeyFamilyRef'].apply(lambda x: x['KeyFamilyID'])
    Texts = pd.DataFrame(r.json()['Structure']['Dataflows']['Dataflow'])['Name'].apply(lambda x: x['#text'])
    return(pd.DataFrame({'DatabaseID': IDs, 
                         'DatabaseText': Texts}))

In [17]:
def dataflow_method():
    base_url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
    service_name = 'Dataflow/'
    request_url = ''.join([base_url, service_name])
    raw_request = requests.get(request_url)
    par_request = pd.DataFrame(raw_request.json()['Structure']['Dataflows']['Dataflow'])
    ids = par_request['KeyFamilyRef'].apply(lambda x: x['KeyFamilyID'])
    texts = par_request['Name'].apply(lambda x: x['#text'])
    return pd.DataFrame({'DatabaseID': ids, 'DatabaseText': texts})


In [18]:
# dfm = DataflowMethod()
dfm = dataflow_method()
dfm.shape

(74, 2)

In [13]:
def DataStructureMethod(database_id, check_query = False):
    
    if check_query:
        available_datasets = DataflowMethod()['DatabaseID'].tolist()
        if not database_id in available_datasets:
            return None

    r = requests.get(''.join(['http://dataservices.imf.org/REST/SDMX_JSON.svc/DataStructure/',database_id]))
    if not r.ok:
        return None

    rparsed = r.json()
    dim_code = pd.Series(rparsed['Structure']['KeyFamilies']['KeyFamily']['Components']['Dimension']).apply(lambda x: x['@codelist']).tolist()
    dim_codedict = [x for x in rparsed['Structure']['CodeLists']['CodeList'] if x['@id'] in dim_code]
    dim_codedict = dict(zip(pd.Series(dim_codedict).apply(lambda x: x['@id']).tolist(), dim_codedict))
    for k in dim_codedict.keys():
        dim_codedict[k] = pd.DataFrame({
                'CodeValue' : pd.Series(dim_codedict[k]['Code']).apply(lambda x: x['@value']),
                'CodeText' : pd.Series(dim_codedict[k]['Code']).apply(lambda x: x['Description']['#text'])
            })
    
    return((dim_code, dim_codedict))

In [21]:
def data_structure_method(database_id, check_query=False):

    if check_query:
        available_datasets = dataflow_method()['DatabaseID'].tolist()
        if database_id not in available_datasets:
            return None

    base_url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
    service_name = 'DataStructure/'
    request_url = ''.join([base_url, service_name, database_id])
    raw_request = requests.get(request_url)

    if not raw_request.ok:
        return None

    rparsed = raw_request.json()['Structure']
    dim_code = pd.Series(rparsed['KeyFamilies']['KeyFamily']['Components']['Dimension']).apply(
        lambda x: x['@codelist']).tolist()
    dim_codedict = [x for x in rparsed['CodeLists']['CodeList'] if x['@id'] in dim_code]
    dim_codedict = dict(zip(pd.Series(dim_codedict).apply(lambda x: x['@id']).tolist(),
                            dim_codedict))

    for k in dim_codedict.keys():
        dim_codedict[k] = pd.DataFrame({
            'CodeValue': pd.Series(dim_codedict[k]['Code']).apply(lambda x: x['@value']),
            'CodeText': pd.Series(
                    dim_codedict[k]['Code']).apply(lambda x: x['Description']['#text'])})

    return (dim_code, dim_codedict)



In [22]:
# dsm = DataStructureMethod('IFS')
dsm = data_structure_method('IFS')
dsm[0]

['CL_FREQ', 'CL_AREA_IFS', 'CL_INDICATOR_IFS']

In [ ]:
def CompactDataMethod(database_id, queryfilter=None, 
                      startdate='2001-01-01', enddate='2001-12-31',
                      checkquery = False, verbose=False):
    
    if checkquery:
        available_datasets = DataflowMethod()['DatabaseID'].tolist()
        if not database_id in available_datasets:
            return None
        
    queryfilterstr = ''
    if len(queryfilter[1]) > 0 and len(queryfilter[0]) == len(queryfilter[1]):
        queryfilterstr = '.'.join(['+'.join(queryfilter[1][k]) for k in queryfilter[0]])
        
    APIstr = ''.join(['http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/',
                      database_id,'/',queryfilterstr,'?startPeriod=',startdate,'&endPeriod=',enddate])
    r = requests.get(APIstr)        
    
    if verbose:
        print('\nmaking API call:\n')
        print(APIstr)
        print('\n')
        
    if not r.ok:
        return None        

    rparsed = r.json()
    
    if not 'Series' in list(rparsed['CompactData']['DataSet'].keys()):
        print('no data available \n')
        return None

    return_df = [x for x in rparsed['CompactData']['DataSet']['Series'] if 'Obs' in x]
    for x in return_df:
        if isinstance(x['Obs'],dict):
            x['Obs'] = [x['Obs']]

    return_df = pd.concat([pd.DataFrame(x) for x in return_df])
    obs_keys = return_df['Obs'].apply(lambda x: [y for y in x.keys()]).tolist()
    obs_keys = set([val for sublist in obs_keys for val in sublist])

    for obs_key in obs_keys:
        return_df[obs_key] = return_df['Obs'].apply(lambda x: x[obs_key] if obs_key in x.keys() else None)

    del return_df['Obs']
    return_df = return_df.reset_index()       

    return(return_df)


In [ ]:
def compact_data_method(database_id, queryfilter=None, startdate='2001-01-01',
                        enddate='2001-12-31', checkquery=False, verbose=False):
    if checkquery:
        available_datasets = dataflow_method()['DatabaseID'].tolist()
        if database_id not in available_datasets:
            return None

    queryfilterstr = ''
    if len(queryfilter[1]) > 0 and len(queryfilter[0]) == len(queryfilter[1]):
        queryfilterstr = '.'.join(['+'.join(queryfilter[1][k]) for k in queryfilter[0]])

    base_url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
    service_name = 'CompactData/'
    request_url = ''.join([base_url, service_name, queryfilterstr, 
                           '?startPeriod=', startdate, '&endPeriod=', enddate])
    raw_request = requests.get(request_url)

    if verbose:
        print('\nmaking API call:\n')
        print(api_str)
        print('\n')

    if not raw_request.ok:
        return None
    rparsed = raw_request.json()

    if 'Series' not in list(rparsed['CompactData']['DataSet'].keys()):
        print('no data available \n')
        return None

    return_df = [x for x in rparsed['CompactData']['DataSet']['Series'] if 'Obs' in x]
    for x in return_df:
        if isinstance(x['Obs'], dict):
            x['Obs'] = [x['Obs']]

    return_df = pd.concat([pd.DataFrame(x) for x in return_df])
    obs_keys = return_df['Obs'].apply(lambda x: [y for y in x.keys()]).tolist()
    obs_keys = set([val for sublist in obs_keys for val in sublist])

    for obs_key in obs_keys:
        return_df[obs_key] = return_df['Obs'].apply(
            lambda x: x[obs_key] if (obs_key in x.keys()) else None)

    del return_df['Obs']
    return_df = return_df.reset_index()

    return return_df

In [ ]:
queryfilter = (['CL_FREA','CL_AREA_IFS','CL_INDICATOR_IFS'], 
               {'CL_FREA':['Q','A'],
                'CL_AREA_IFS':['GR'],
                'CL_INDICATOR_IFS':['NGDP_EUR','NGDP_XDC']})
CompactDataMethod('IFS', queryfilter)

In [ ]:
# teststr = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/IFS/Q+A.GR.NGDP_EUR+NGDP_XDC?startPeriod=2001-01-01&endPeriod=2001-12-31'
teststr = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/IFS/A.GR.NGDP_EUR+NGDP_XDC?startPeriod=1990-01-01&endPeriod=2001-12-31'
# teststr = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/IFS/Q+A.GR.NGDP_EsUR+NGDP_sXDC?startPeriod=2001-01-01&endPeriod=2001-12-31'

In [ ]:
r = requests.get(teststr)        
rparsed = r.json()
return_df = [x for x in rparsed['CompactData']['DataSet']['Series'] if 'Obs' in x]
for x in return_df:
    if isinstance(x['Obs'],dict):
        x['Obs'] = [x['Obs']]
        
return_df = pd.concat([pd.DataFrame(x) for x in return_df])
obs_keys = return_df['Obs'].apply(lambda x: [y for y in x.keys()]).tolist()
obs_keys = set([val for sublist in obs_keys for val in sublist])

for obs_key in obs_keys:
    return_df[obs_key] = return_df['Obs'].apply(lambda x: x[obs_key] if obs_key in x.keys() else None)

del return_df['Obs']
return_df = return_df.reset_index()

In [ ]:
return_df.reset_index()

In [ ]:
return_df['Obs'].iloc[11]['@OBS_VALUE']

In [ ]:
return_df['Obs'].appl

In [ ]:
rparsed['CompactData']['DataSet']['Series'][0]

In [ ]:
adict = {'a':1, 'b':2, 'c':3}

In [ ]:
'a' in adict

In [ ]:
adict.iteritems()